
# **MINI PROJECT in bioinformatics: part 1 (with changes for part 2)**

## Data processing:

First, initialize the data structures below:

**original_data_dict**: dictionary that stores the lines from the data files

- key= genome name

- value= list of words of the current genome (each word is a list of cogs)


**gen_uid_dict:**
- key = genome name
- value = uid


In [ ]:
import sys



**process_data function**:

-for each line in the file we insert to **original_data_dict** by the genome_NC.

-insert the genome_NC and uid into **gen_uid_dict**




In [ ]:
def process_data(file, gen_uid_dict, original_data_dict):
    with open(file) as my_file:
        for line in my_file:
            splited = line.split("#")
            genome_number = splited[1]
            line_of_uid = splited[-2]
            splited_uid_line = line_of_uid.split("_")
            my_uid = splited_uid_line[-1]
            word_segment = splited[-1]
            gen_uid_dict[genome_number] = my_uid
            splited_word = word_segment.split("\t")
            without_first = splited_word[1:-1]

            if genome_number in original_data_dict:
                original_data_dict[genome_number].append(without_first)
            else:
                my_list = [without_first]
                original_data_dict[genome_number] = my_list
    my_file.close

orig_stdout = sys.stdout


##**find clusters(s,d,q) function:**

Given a set of COG-spelled genomes S, where each
genome is segmented into segments such that each segment could contain one or
more operons of the genomes in the database
Find all gene
clusters 𝑤 (over the COG alphabet) of length 𝑑 that are conserved in ≥ 𝑞 of the
genomes in the database 𝑆.

S = original_data_dict
**all_clusters_dict:**

- key = cluster (set of cogs)

- value: an **inner dicitionary**: key = order of cogs,
value = list of genomes numbers


**final_clusters_dict:** (clusters that appears at least q times)

- key = cluster (set of cogs)

- value: an **inner dicitionary**: key = order of cogs,
value = list of genomes numbers



**genome_substrings_dict:** (for each genome we store clusters and the first order we found)

- key = cluster: set of cogs
- value = order of cogs


**clusters_counters_dict:** (number of appearnces in all genomes for each cluster)

- key = cluster: set of cogs

- value = number of appearnces


In [ ]:
def find_clusters(s, d, q, final_clusters_dict, all_clusters_dict, clusters_counter_dict):
    for curr_genome in s:
        genome_substrings_dict = {}
        for segment in s[curr_genome]: #iterate over different lines of the same genome
            for i in range(len(segment) - d+1):#sliding window
                curr_d_mer = segment[i:i + d] #create the current dmer
                temp_dict = {}
                my_str_order = ""
                if 'X' not in curr_d_mer and len(curr_d_mer)==d: #ignore dmers that contains X
                    cluster_set = set()
                    for cog in curr_d_mer:# add each cog to set represent the cluster
                        my_str_order += cog
                        if cog != '':
                            cluster_set.add(cog)
                    frozen_cluster = frozenset(cluster_set)
                    if len(frozen_cluster) == d:#make sure the set in size d (can be cases cog is repeated and size of set is smaller)
                        if frozen_cluster not in genome_substrings_dict: #found a new cluster
                            genome_substrings_dict[frozen_cluster] = my_str_order

        for cluster in genome_substrings_dict:
            my_hashmap = {}
            curr_order = genome_substrings_dict[cluster]
            #maintain the number of appearance for each cluster
            if cluster in clusters_counter_dict:
                clusters_counter_dict[cluster] += 1
            else:
                clusters_counter_dict[cluster] = 1
            #add cluster to clusters dictionary and then add genome number to the list of the cluster by order
            if cluster not in all_clusters_dict:
                my_hashmap[curr_order] = [curr_genome]
                all_clusters_dict[cluster] = my_hashmap
            else: #cluster in dict
                curr_cluster = all_clusters_dict[cluster]
                if curr_order in curr_cluster:
                    curr_cluster[curr_order].append(curr_genome)
                else:
                    curr_cluster[curr_order] = [curr_genome]
    #add to final_clusters_dict clusters that appear at least q times

    #change clusters_counter_dict to include only clusters that apears at least q times
    clusters_counter_dict = {cluster: value for cluster, value in clusters_counter_dict.items() if value >= q}
    sorted_list = sorted(clusters_counter_dict, key=lambda key: clusters_counter_dict[key], reverse=True)
    sys.stdout = orig_stdout

    return sorted_list

##**profile by gene cluster:**
subroutine that, given a gene cluster, computes and prints a descriptive “profile” of the cluster. For each cluster we report, the output include a “characterization profile” including the following information:
_____________________________________________________________________
1) The rank of the gene cluster in the sorted list of all gene clusters found for the given parameters (d,q) and the input genome file and the number of instances it has among the genomes in the input file.

**create_clusters_rank_dict function:**

create the **rank_dict** that contain : key = cluster, value = rank of the cluster.
______________________________________________________________________
2) Pattern information: Print the set of COGs participating in the pattern of the gene cluster

**create_cog_dict function:**

create the **cog_dic** that contain: key = cog, value = line from "COG_INFO_TABLE"

**get_pattern_info(cluster):**

for each cog in cluster set, prints the coresponding information from **cog_dict**

_______________________________________________________________________
3) Report the various gene orders of the pattern in the instances and the percentage of instances found per each gene order

 **get_various_gene_orders(cluster):**

get the gene orders from **final_clusters_dict[cluster]** and calculate the percentage.

 For each of the distinct orders in which the gene cluster was found, report distribution statistics of its abundance in phyla and habitats:

 A. Report the various phyla and the percentage of instances per this order found in each phylum.





 B. Report the environmental descriptions of the instances of the gene cluster in which the order is found and the percentage of instances of the gene cluster in which the order is found per each environmental category.

**create_taxa_dict function:**

create the **taxa_dict**: key = uid, value = phylum





**create_habitat_dict function:**

create the **habitat_dict** : key = uid, value = phylum

**get_taxa(cluster):**
- for each order found for the cluster, we iterate over genes, get the phyla using the **taxa_dict** by gene uid (gen_uid_dict[genome]).

- calculate the percent of each phylum, by maintaining a dictionary called **phylas**, that counts the amount of the different phylums.


**get_habitat(cluster):**
- for each order found for the cluster, we iterate over genes, get the habitat using the **habitat_dict** by gene uid (gen_uid_dict[genome]).

- calculate the percent of each habitat, by maintaining a dictionary called **habitats**, that counts the amount of the different habitats.


##**get_cluster_profile function:**

for the given cluster, all the requested information is printed, by calling the **get** functions above

In [ ]:

def create_clusters_rank_dict(clusters_counter_dict, rank_dict): # C1
    sorted_list = sorted(clusters_counter_dict, key=lambda key: clusters_counter_dict[key], reverse= True)
    for i in range(len(sorted_list)):
        rank_dict[sorted_list[i]] = i+1
    return rank_dict


def create_cog_dict(cog_dict):  # C2
    with open("COG_INFO_TABLE.txt",'r', encoding ='latin-1') as cog_file:
        data_cogs_lines = cog_file.readlines()
        for cog_line in data_cogs_lines:
            splited_line = cog_line.split(";")
            cog_number = splited_line[0]
            cog_dict[cog_number] = cog_line
    cog_file.close

def get_pattern_info(cluster,cog_dict):
    sorted_cluster = sorted(map(int, set(cluster)))#sort cogs
    for cog in sorted_cluster:
        my_cog_str = str(cog)
        #add zeros to start of cog if needed
        to_add = 4-len(str(cog))
        for i in range(to_add):
            my_cog_str = '0'+ my_cog_str
        key = "COG" + my_cog_str
        print(my_cog_str, cog_dict[key], end="")


def create_taxa_dict(taxa_dict):  # C3A
    with open("taxa.txt") as taxa_file:
        lines = taxa_file.readlines()
        for i in range(1, len(lines)):
            splited = lines[i].split(",")
            uid_line = splited[-4]
            phylum = splited[1]
            uid_word = uid_line.split("_")
            uid = uid_word[-1]
            taxa_dict[uid] = phylum
    taxa_file.close


def create_habitat_dict(habitat_dict):  # C3A
    with open("habitat.txt") as habitat_file:
        lines = habitat_file.readlines()
        for i in range(1, len(lines)):
            splited = lines[i].split(";")
            my_habitat = splited[-1]
            uid_line = splited[1]
            splited_uid = uid_line.split("_")
            uid = splited_uid[-1]
            habitat_dict[uid] = my_habitat
    habitat_file.close


def get_taxa(cluster, final_clusters_dict, taxa_dict, gen_uid_dict):
    cluster_dict = final_clusters_dict[cluster]
    for order in cluster_dict:
        phylas = {}
        for genome in cluster_dict[order]:
            my_uid = gen_uid_dict[genome]
            my_phyla = taxa_dict[my_uid]
            if my_phyla in phylas:
                phylas[my_phyla] +=1
            else:
                phylas[my_phyla] = 1
        total = len(cluster_dict[order])
        print("order: ", order)
        for phyla in phylas:
            percent = round(phylas[phyla] / total * 100, 2)
            print("phylum: ", phyla, end=" ")
            print("percentage: ", percent, "%")


def get_habitat(cluster, final_clusters_dict, habitat_dict, gen_uid_dict):
    cluster_dict = final_clusters_dict[cluster]
    for order in cluster_dict:
        habitats = {}
        for genome in cluster_dict[order]:
            my_habitat = habitat_dict[gen_uid_dict[genome]]
            if my_habitat in habitats:
                habitats[my_habitat] +=1
            else:
                habitats[my_habitat] = 1
        total = len(cluster_dict[order])
        print("order: ", order)
        for habitat in habitats:
            percent = round(habitats[habitat] / total * 100, 2)
            print("environment: ", habitat, end="")
            print("percentage: ", percent, "%")


def get_various_gene_orders(cluster,final_clusters_dict, clusters_counter_dict):
    my_dict = final_clusters_dict[cluster]
    total = clusters_counter_dict[cluster]
    for order in my_dict:
        percent = round(len(my_dict[order]) / total * 100, 2)
        print(order, "percentage of order: ", percent, "%")


def get_cluster_profile(cluster, rank_dict, clusters_counter_dict, final_clusters_dict, gen_uid_dict, taxa_dict, habitat_dict,cog_dict):
    print("Cluster: ")
    #sort set of cluster by COG id
    sorted_cluster = sorted(map(int, set(cluster)))
    cogs_list = []
    for cog in sorted_cluster:
        my_cog_str = str(cog)
        to_add = 4-len(str(cog))
        for i in range(to_add):
            my_cog_str = '0'+ my_cog_str
        cogs_list.append(my_cog_str)
    print(cogs_list)
    print("Rank: ", rank_dict[cluster])
    print("Appearance amount:", clusters_counter_dict[cluster])
    print("")
    print("Pattern info: ")
    get_pattern_info(cluster,cog_dict)
    print("")
    print("Instance information: ")
    print("")
    print("Taxonomical info: ")
    get_taxa(cluster, final_clusters_dict, taxa_dict, gen_uid_dict)
    print("")
    print("Various gene_order: ")
    get_various_gene_orders(cluster, final_clusters_dict, clusters_counter_dict)
    print("")
    print("environmental description: ")
    get_habitat(cluster, final_clusters_dict, habitat_dict, gen_uid_dict)
    print("")

##**Run the  program**

 extract all gene clusters of cardinality ranging from d=2 up to
d=4, using q=5 from the file of plasmid genomes, and for each cardinality value,
report the 4 most abundant gene clusters found, sorted by decreasing abundance
order per each value of d.
Print the reported gene clusters into an external output file, producing a separate ourtput for each cardinality value d, sorted in decreasing order by the number of instances per each gene cluster.


In [ ]:
def run_program(data_file):
    original_data_dict = {}
    all_clusters_dict = {}
    final_clusters_dict = {}
    gen_uid_dict = {}
    clusters_counter_dict = {}
    rank_dict = {}
    cog_dict = {}
    taxa_dict = {}
    habitat_dict = {}

    orig_stdout = sys.stdout
    process_data(data_file, gen_uid_dict, original_data_dict)
    create_cog_dict(cog_dict)
    create_taxa_dict(taxa_dict)
    create_habitat_dict(habitat_dict)


    for d_val in range(2, 5):
        all_clusters_dict = {}
        final_clusters_dict = {}
        clusters_counter_dict = {}
        rank_dict = {}
        find_clusters(original_data_dict, d_val, 5, final_clusters_dict, all_clusters_dict, clusters_counter_dict)
        final_clusters_dict = {cluster: values for cluster, values in all_clusters_dict.items() if clusters_counter_dict[cluster] >= 5}
        curr_rank_dict = create_clusters_rank_dict(clusters_counter_dict, rank_dict)#find rank of the diffrent clusters
        keys = [key for key in curr_rank_dict]#make a list of clusters only from rank_dict
        my_ranks = keys[:4] #The 4 most abundant clusters
        file_name = "gene_clusters_report_d=%s.txt" % d_val
        with open(file_name, 'w') as output_file:
            sys.stdout = output_file# change the standart output to print into the new file
            index = 1
            for cluster in my_ranks:
                get_cluster_profile(cluster, rank_dict, clusters_counter_dict, final_clusters_dict, gen_uid_dict, taxa_dict, habitat_dict,cog_dict)
                index +=1
            output_file.close()
        sys.stdout = orig_stdout #return the standard output


run_program("cog_words_plasmid.txt")


# **Part 2**
**Data Preparation:** “Clean” the file of bacterial genomes so that only genomes that have at
least one plasmid in the data remain


In [ ]:
def clean_bac_genomes_file():
    uid_from_plasmid = {}
    with open("cog_words_plasmid.txt", 'r') as plasmid_file:
        for line in plasmid_file:
            splited = line.split('#')
            uid_line = splited[3]
            splited_uid_line = uid_line.split('_')
            #print(splited_uid_line)
            uid = splited_uid_line[-1]
            uid_from_plasmid.update({uid:""})

    with open('clean_file.txt', 'w') as clean_file:
        with open("cog_words_bac.txt",'r') as bac_file:
            for line in bac_file:
                splited = line.split('#')
                uid_line = splited[3]
                splited_uid_line = uid_line.split('_')
                uid = splited_uid_line[-1]
                if uid in uid_from_plasmid:
                    clean_file.write(line)


clean_bac_genomes_file()

**our comparative analysis approach for gene cluster profiles:**

We Propose a novel approach to compare two gene cluster profiles (we focus in case of the same cluster in 2 different  datasets) and produce a score (a number between 0 and 100) to measure how distinct the clusters are.
the calculation is composed from the following parameters:


1) **X (Genetic Variation Score):** This parameter evaluates the difference in abundance between various gene orders within the clusters. We utilize a formula that directly computes the absolute difference between the gene order percentages of the two profiles. For instance:

    - Profile 1: [98.06%, 1.94%]
    - Profile 2: [80%, 20%]

   
The calculation for **X** involves:

(| 98.06 - 80 | + | 1.94 - 20 |) / 2

2) **Y (Taxa Score):** This parameter assesses the dissimilarity in taxa parameters between the two datasets, while considering the percentages of different gene orders. Similar to **X**, we directly compute the absolute difference between the taxonomic composition percentages of the two profiles.

3) **Z (Environment Score):** Here, we quantify the discrepancy in environmental parameters between the two datasets, while also factoring in the percentages of different gene orders. Like **X** and **Y**, the calculation involves directly computing the absolute difference between the environment percentages of the two profiles.

The **final_score** is computed as the average of the three individual scores(X,Y,Z)


This approach provides a straightforward assessment of the dissimilarity between gene cluster profiles, focusing solely on the absolute difference. The final score facilitates comparative analysis and aids in identifying clusters with notable distinctions across datasets.


In [ ]:
import math
various_gene_orders = {}
taxa_info = {}
env_info = {}
def calculate_profile_score(cluster, q1,q2):
    global various_gene_orders, taxa_info, env_info

    num_of_orders = 0
    max = 0
    x = 0

    for order in various_gene_orders:
        x += math.sqrt((various_gene_orders[order][0]-various_gene_orders[order][1]) ** 2)

    x = round(x / 2)

    y = 0

    for order in taxa_info :
        i = 0
        data = taxa_info[order]

        for item in data:
            my_list = data[item]
            y += math.sqrt(((my_list[0]*various_gene_orders[order][0]/100 )-(my_list[1]*various_gene_orders[order][1]/100))** 2)


    y = round(y / 2)

    #print(y)

    z = 0

    for order in env_info:
        i = 0
        data = env_info[order]

        for item in data:
            my_list = data[item]
            z += math.sqrt((my_list[0]*various_gene_orders[order][0]/100 - my_list[1]*various_gene_orders[order][1]/100) ** 2)


    z = round(z / 2)

    if x > 100 or y > 100 or z > 100:
        print("x: ", x,"y: ",y,"z: ", z)
    score = (x + y + z) / 3
    return score


def create_cluster_profile(s, cluster, taxa_info, env_info, various_gene_orders,index,q):
    gen_uid_dict = {}
    original_data_dict = {}
    cog_dict = {}
    taxa_dict = {}
    habitat_dict = {}
    final_clusters_dict = {}
    all_clusters_dict = {}
    clusters_counter_dict = {}
    process_data(s, gen_uid_dict, original_data_dict)
    create_cog_dict(cog_dict)
    create_taxa_dict(taxa_dict)
    create_habitat_dict(habitat_dict)
    d_val = len(cluster)
    find_clusters(original_data_dict, d_val, 1, final_clusters_dict, all_clusters_dict, clusters_counter_dict) # q=2 is a default value
    final_clusters_dict = {cluster: values for cluster, values in all_clusters_dict.items() if clusters_counter_dict[cluster] >= q}
    get_gene_orders_dict(cluster,index, final_clusters_dict, clusters_counter_dict)
    get_taxa_info(cluster,index, final_clusters_dict, taxa_dict, gen_uid_dict)
    get_env_info(cluster,index, final_clusters_dict, gen_uid_dict, habitat_dict)
    print(clusters_counter_dict[cluster])


def get_gene_orders_dict(cluster,index, final_clusters_dict, clusters_counter_dict):
    global various_gene_orders
    my_dict = final_clusters_dict[cluster]
    total = clusters_counter_dict[cluster]
    for order in my_dict:
        percent = round(len(my_dict[order]) / total * 100, 2)
        if order not in various_gene_orders:
            various_gene_orders[order] = [0,0]
            various_gene_orders[order][index] = percent
        else:
            various_gene_orders[order][index] = percent


def get_taxa_info(cluster, index, final_clusters_dict, taxa_dict, gen_uid_dict):
    global taxa_info
    cluster_dict = final_clusters_dict[cluster]
    for order in cluster_dict:
        phylas = {}
        for genome in cluster_dict[order]:
            my_uid = gen_uid_dict[genome]
            my_phyla = taxa_dict[my_uid]
            if my_phyla in phylas:
                phylas[my_phyla] += 1
            else:
                phylas[my_phyla] = 1
        total = len(cluster_dict[order])
       #-----------------------------------
        if order not in taxa_info:
            taxa_info[order] = {}
        curr_order_dict = taxa_info[order]
        for phyla in phylas:
            percent = round(phylas[phyla] / total * 100, 2)
            if phyla not in curr_order_dict:
                curr_order_dict[phyla] = [0,0]
                curr_order_dict[phyla][index] = percent
            else:
                curr_order_dict[phyla][index] = percent


def get_env_info(cluster,index, final_clusters_dict, gen_uid_dict, habitat_dict):
    global env_info
    cluster_dict = final_clusters_dict[cluster]
    for order in cluster_dict:
        habitats = {}
        for genome in cluster_dict[order]:
            my_uid = gen_uid_dict[genome]
            my_habitat = habitat_dict[my_uid]
            if my_habitat in habitats:
                habitats[my_habitat] += 1
            else:
                habitats[my_habitat] = 1
            total = len(cluster_dict[order])
            if order not in env_info:
                env_info[order] = {}
            curr_order_dict = env_info[order]
            for habitat in habitats:
                percent = round(habitats[habitat] / total * 100, 2)
                if habitat not in curr_order_dict:
                    curr_order_dict[habitat] = [0,0]
                    curr_order_dict[habitat][index] = percent
                else:
                    curr_order_dict[habitat][index] = percent


def test_score():
    global taxa_info
    global env_info
    global various_gene_orders
    various_gene_orders = {}
    taxa_info = {}
    env_info = {}
    cluster_set = frozenset({'1898', '1088', '1209', '1091'})
    create_cluster_profile("cog_words_plasmid.txt", cluster_set, taxa_info, env_info, various_gene_orders,0,1)
    create_cluster_profile("clean_file.txt", cluster_set, taxa_info, env_info, various_gene_orders,1,10)
    print(calculate_profile_score(cluster_set,1,10))


#test_score()
#the target of this function is to check whether the function calculate_score() is working correctly

13
34
69.33333333333333
The order:  1088120918981091
Percentage of the order in the first set:  7.69
Percentage of the order in the second set:  14.71
The order:  1898108812091091
Percentage of the order in the first set:  7.69
Percentage of the order in the second set:  0
The order:  1898108810911209
Percentage of the order in the first set:  46.15
Percentage of the order in the second set:  14.71
The order:  1898109112091088
Percentage of the order in the first set:  7.69
Percentage of the order in the second set:  0
The order:  1209189810911088
Percentage of the order in the first set:  7.69
Percentage of the order in the second set:  2.94
The order:  1088109118981209
Percentage of the order in the first set:  7.69
Percentage of the order in the second set:  0
The order:  1209189810881091
Percentage of the order in the first set:  15.38
Percentage of the order in the second set:  47.06
The order:  1088109112091898
Percentage of the order in the first set:  0
Percentage of the order 

**3)** Based on your code from Part 1, write a program\pipeline that solves the following
problem in comparative analysis of gene cluster characterization profiles:

Given two COG-spelled sets of genomes S1 and S2, each in a separate file, and also given parameters:

**d** – the number of genes in the sought gene cluster

**q1** – the required quorum for a gene cluster in genome set S1 (for example, the plasmid genome set).

**q2**- the required quorum for a gene cluster in genome set S2 (for example, the chromosome genome set).

Objective: Find a gene-cluster GC of cardinality d that appears in at least q1 of the genomes in set S1, and also appears in at least q2 genomes of Set S2, such that the following requirement is obeyed: the characterization profile of gene cluster GC in set S1 differs significantly from the characterization profile of the same gene cluster in set S2.

In [ ]:
import copy

def find_GC(s1,s2,d,q1,q2):
    global env_info
    global taxa_info
    global various_gene_orders
    gen_uid_dict_s1 = {}
    original_data_dict_s1 = {}
    cog_dict_s1 = {}
    taxa_dict_s1 = {}
    habitat_dict_s1 = {}
    final_clusters_dict_s1 = {}
    all_clusters_dict_s1 = {}
    clusters_counter_dict_s1= {}
    process_data(s1, gen_uid_dict_s1, original_data_dict_s1)
    create_cog_dict(cog_dict_s1)
    create_taxa_dict(taxa_dict_s1)
    create_habitat_dict(habitat_dict_s1)
    find_clusters(original_data_dict_s1, d, q1, final_clusters_dict_s1, all_clusters_dict_s1, clusters_counter_dict_s1) # q=2 is a default value
    final_clusters_dict_s1 = {cluster: values for cluster, values in all_clusters_dict_s1.items() if clusters_counter_dict_s1[cluster] >= q1}
    clusters_counter_dict_s1 = {cluster: value for cluster, value in clusters_counter_dict_s1.items() if value >= q1}

    gen_uid_dict_s2 = {}
    original_data_dict_s2 = {}
    cog_dict_s2 = {}
    taxa_dict_s2 = {}
    habitat_dict_s2 = {}
    final_clusters_dict_s2 = {}
    all_clusters_dict_s2 = {}
    clusters_counter_dict_s2= {}
    process_data(s2, gen_uid_dict_s2, original_data_dict_s2)
    create_cog_dict(cog_dict_s2)
    create_taxa_dict(taxa_dict_s2)
    create_habitat_dict(habitat_dict_s2)
    find_clusters(original_data_dict_s2, d, q2, final_clusters_dict_s2, all_clusters_dict_s2, clusters_counter_dict_s2) # q=2 is a default value
    final_clusters_dict_s2 = {cluster: values for cluster, values in all_clusters_dict_s2.items() if clusters_counter_dict_s2[cluster] >= q2}
    clusters_counter_dict_s2 = {cluster: value for cluster, value in clusters_counter_dict_s2.items() if value >= q2}
    score = 0
    GC = {}
    for cluster in final_clusters_dict_s1:
        score = 0
        env_info = {}
        taxa_info = {}
        various_gene_orders = {}
        if cluster in final_clusters_dict_s2:
            #print(cluster)
            get_gene_orders_dict(cluster, 0, final_clusters_dict_s1, clusters_counter_dict_s1)
            get_taxa_info(cluster, 0, final_clusters_dict_s1, taxa_dict_s1, gen_uid_dict_s1)
            get_env_info(cluster,0, final_clusters_dict_s1, gen_uid_dict_s1, habitat_dict_s1)
            get_gene_orders_dict(cluster, 1, final_clusters_dict_s2, clusters_counter_dict_s2)
            get_taxa_info(cluster,1, final_clusters_dict_s2, taxa_dict_s2, gen_uid_dict_s2)
            get_env_info(cluster, 1, final_clusters_dict_s2, gen_uid_dict_s2, habitat_dict_s2)
            score = calculate_profile_score(cluster, q1,q2)
        if score > 60 :
            GC = set(cluster)
            break
    return  [GC, score]

def test_GC():
    print(find_GC("cog_words_plasmid.txt","clean_file.txt",2,2,2))

test_GC()


[{'4587', '3694'}, 60.333333333333336]


**4)** Experiment with your program on the data files (i.e., run benchmarks with different
values of d, q1, q2 and examine the results) till you find an interesting result

In [ ]:
def experiment_GC_values():
    max_score = 0
    best_d = 0
    best_q1 = 0
    best_q2 = 0
    for d_val in range(2,6):
        for q1_val in range(1,100):
            q2_val = q1_val * 10
            gen_cluster_list = find_GC("cog_words_plasmid.txt","clean_file.txt",d_val,q1_val,q2_val)
            my_score = gen_cluster_list[1]
            if my_score > max_score:
                max_score = my_score
                best_d = d_val
                best_q1 = q1_val
                best_q2 = q2_val
                my_list = gen_cluster_list
                print(my_list, best_d, best_q1, best_q2) #test
    print(my_list, best_d, best_q1, best_q2)

#experiment_GC_values()

**Process the data about the cluster**

The cluster: {'1898', '1088', '1209', '1091'}

(d: 4, q1: 1, q2: 10)

cog information:

| COG       | Functional Category         | Role                               | Enzyme                                         |
|-----------|-----------------------------|------------------------------------|------------------------------------------------|
| COG1898   | Cellular Processes and Signaling | Cell wall/membrane/envelope biogenesis | dTDP-4-dehydrorhamnose 3,5-epimerase or related enzyme |
| COG1088   | Cellular Processes and Signaling | Cell wall/membrane/envelope biogenesis | dTDP-D-glucose 4,6-dehydratase                    |
| COG1209   | Cellular Processes and Signaling | Cell wall/membrane/envelope biogenesis | dTDP-glucose pyrophosphorylase                    |
| COG1091   | Cellular Processes and Signaling | Cell wall/membrane/envelope biogenesis | dTDP-4-dehydrorhamnose reductase                 |


 These enzymes are interconnected through their involvement in common metabolic pathways and the shared substrates and products in their respective reactions. Let's break down the connections:

# Enzyme Information

## 1. COG1898: dTDP-4-dehydrorhamnose 3,5-epimerase
- **Reaction:** dTDP-4-dehydro-6-deoxy-D-glucose ⇌ dTDP-4-dehydro-6-deoxy-L-mannose

## 2. COG1088: dTDP-glucose 4,6-dehydratase
- **Reaction:** dTDP-glucose ⇌ dTDP-4-dehydro-6-deoxy-D-glucose + H2O
- **Connection to COG1898:** COG1088 produces a substrate (dTDP-4-dehydro-6-deoxy-D-glucose) for COG1898.

## 3. COG1209: Glucose-1-phosphate thymidylyltransferase
- **Reaction:** dTTP + alpha-D-glucose 1-phosphate ⇌ diphosphate + dTDP-glucose
- **Family:** Transferases (nucleotidyltransferases)
- **Pathways:** Nucleotide sugars metabolism, streptomycin biosynthesis, polyketide sugar unit biosynthesis.
- **Connection to COG1088:** COG1209 produces dTDP-glucose, a substrate for COG1088.

## 4. COG1091: dTDP-4-dehydrorhamnose reductase
- **Reaction:** dTDP-6-deoxy-L-mannose + NADP+ ⇌ dTDP-4-dehydro-6-deoxy-L-mannose + NADPH + H+
- **Connection to COG1898:** COG1091 shares a product (dTDP-4-dehydro-6-deoxy-L-mannose) with COG1898.




In [ ]:
def get_cluster_profile(cluster, q1,q2):
    global various_gene_orders, taxa_info, env_info
    plasmid_curr = 13
    chrom_curr = 17
    num_of_orders = 0
    index = 0

    for order in various_gene_orders:
        index = index + 1
        print(index,".order: ",order)
        print("")
        print("p: ", round(various_gene_orders[order][0]),"%,",round(various_gene_orders[order][0]/100 *plasmid_curr) )
        print("c: ", round(various_gene_orders[order][1]),"%,", round(various_gene_orders[order][1]/100*chrom_curr))
        print("")
        data = taxa_info[order]
        print("phylum")
        for item in data:
            my_list = data[item]
            print(item)
            print("p: ", round(my_list[0]*various_gene_orders[order][0]/100),"%, ",round((my_list[0]*various_gene_orders[order][0]/100)*plasmid_curr/100))
            print("c: ",round( my_list[1]*various_gene_orders[order][1]/100),"%, ",round(chrom_curr*(my_list[1]*various_gene_orders[order][1]/100)/100))
            print("")
        data = env_info[order]
        print("habitats")
        for item in data:
            my_list = data[item]
            print(item)
            print("p: ", round(my_list[0]*various_gene_orders[order][0]/100),"%,",round((my_list[0]*various_gene_orders[order][0]/100)*plasmid_curr/100))
            print("c: ", round(my_list[1]*various_gene_orders[order][1]/100),"%,",round(chrom_curr*(my_list[1]*various_gene_orders[order][1]/100)/100))
            print("")
from tabulate import tabulate


def get_cluster_profile_1(cluster, q1, q2):
    global various_gene_orders, taxa_info, env_info
    plasmid_curr = 13
    chrom_curr = 17
    num_of_orders = 0
    index = 0

    table_data = []

    for order in various_gene_orders:
        index = index + 1
        order_data = []
        order_data.append(f"{index}. order: {order}")

        p_percentage = round(various_gene_orders[order][0])
        c_percentage = round(various_gene_orders[order][1])

        # Check if both plasmid and chromosome percentages are non-zero
        if p_percentage != 0 or c_percentage != 0:
            order_p_value = round(p_percentage / 100 * plasmid_curr)
            order_c_value = round(c_percentage / 100 * chrom_curr)

            order_data.append(f"{p_percentage}%, {order_p_value}")
            order_data.append(f"{c_percentage}%, {order_c_value}")
            if order_p_value != 0 or order_c_value != 0:
                table_data.append(order_data)
                table_data.append(["-----------------------", "-----------------------", "-----------------------"])
            data = taxa_info[order]
            for item in data:
                my_list = data[item]
                p_phylum = round(my_list[0] * various_gene_orders[order][0] / 100)
                c_phylum = round(my_list[1] * various_gene_orders[order][1] / 100)

                # Check if both plasmid and chromosome percentages are non-zero
                if p_phylum != 0 or c_phylum != 0:
                    phylum_p_value = round(p_phylum * plasmid_curr / 100)
                    phylum_c_value = round(c_phylum * chrom_curr / 100)
                    phylum_data = [item, f"{p_phylum}%, {phylum_p_value}", f"{c_phylum}%, {phylum_c_value}"]
                    if phylum_p_value != 0 or phylum_c_value != 0:
                        table_data.append(phylum_data)
            table_data.append(["-----------------------", "-----------------------", "-----------------------"])
            data = env_info[order]
            for item in data:
                my_list = data[item]
                p_habitat = round(my_list[0] * various_gene_orders[order][0] / 100)
                c_habitat = round(my_list[1] * various_gene_orders[order][1] / 100)

                # Check if both plasmid and chromosome percentages are non-zero
                if p_habitat != 0 or c_habitat != 0:
                    habitat_p_value = round(p_habitat * plasmid_curr / 100)
                    habitat_c_value = round(c_habitat * chrom_curr / 100)
                    habitat_data = [item, f"{p_habitat}%, {habitat_p_value}", f"{c_habitat}%, {habitat_c_value}"]
                    if habitat_p_value != 0 or habitat_c_value != 0:
                        table_data.append(habitat_data)
            table_data.append(["________________________", "________________________", "________________________"])
    headers = ["Category", "Plasmid (%)", "Chromosome (%)"]
    print(tabulate(table_data, headers=headers, tablefmt="pretty"))



def print_cluster_profile():
    global taxa_info
    global env_info
    global various_gene_orders
    various_gene_orders = {}
    taxa_info = {}
    env_info = {}
    cluster_set = frozenset({'1898', '1088', '1209', '1091'})
    create_cluster_profile("cog_words_plasmid.txt", cluster_set, taxa_info, env_info, various_gene_orders,0,1)
    create_cluster_profile("clean_file.txt", cluster_set, taxa_info, env_info, various_gene_orders,1,10)
    get_cluster_profile_1(cluster_set, 1,10)


print_cluster_profile()

13
159
+----------------------------+--------------------------+--------------------------+
|          Category          |       Plasmid (%)        |      Chromosome (%)      |
+----------------------------+--------------------------+--------------------------+
| 1. order: 1088120918981091 |          8%, 1           |          23%, 4          |
|  -----------------------   | -----------------------  | -----------------------  |
|       Proteobacteria       |          8%, 1           |          21%, 4          |
|  -----------------------   | -----------------------  | -----------------------  |
|       Not Annotated        |          8%, 1           |          3%, 1           |
|     Marine Environment     |          0%, 0           |          3%, 1           |
|     Soil and Sediment      |          0%, 0           |          6%, 1           |
|           Plant            |          0%, 0           |          7%, 1           |
|  ________________________  | ________________________ | 

The genes RfbD, RfbB, RfbC, and RmlA1, identified in the study : **O-Antigen-Dependent Colicin Insensitivity of Uropathogenic Escherichia coli.**
These play roles in the biosynthesis of lipopolysaccharide (LPS), a crucial component of the outer membrane of Gram-negative bacteria. These genes are involved in the O-antigen synthesis pathway.

The O-antigen is the outermost component of the LPS molecule and contributes to the structural and functional properties of the bacterial outer membrane. The study suggests that the density of O-antigen is a critical factor in determining the sensitivity of uropathogenic Escherichia coli (UPEC) to colicins.

The density of the O-antigen, influenced by the activities of these genes, affects the ability of colicins to bind to their receptors and, consequently, the sensitivity of the bacteria to colicin-mediated killing.

After observation of the cluster profile we found, considering the different orders, habitats, and taxa information, we can notice some findings.

There are some data we found only in the plasmid:

•	(order number 2 ,1,   Acidobacteria, Not Annotated)
•	(order number 3 ,1, Proteobacteria, Animal)
•	(order number 4, 1,   Spirochaetes, Animal)
•	(order number 6,1, Firmicutes, Host)

We noticed different distributions of the factors we tested, and it also appears that the gene cluster is more abundant in plasmid relative to chromosomes. Therefore, we suggest several hypotheses of advantages that clusters give in relation to the article and the role of the genes in the biosynthesis of lipopolysaccharides (LPS):
1.	Functionality in Variable Environments: The prevalence of LPS biosynthesis genes on plasmids, observed across various habitats (e.g. Acidobacteria, Proteobacteria, Spirochaetes, and Firmicutes), suggests an adaptive advantage for bacteria in fluctuating environments. Plasmids enable rapid modulation of cell surface structures, crucial for survival in diverse conditions.
2.	Horizontal Gene Transfer (HGT): The consistent presence of these genes on plasmids indicates their potential for horizontal gene transfer (HGT) among bacterial populations. This genetic exchange allows for the acquisition of traits related to outer membrane composition and colicin insensitivity across different taxa.
3.	Ecological Niche Adaptation: The data showing plasmid localization of LPS biosynthesis genes in taxa associated with specific ecological niches (e.g. Animal ) suggests their role in niche adaptation. This localization likely enhances bacterial fitness within these niches, possibly through evasion of host immune responses.
4.	Selective Pressure and Fitness Cost: Despite the fitness cost associated with plasmid maintenance, the widespread distribution of these genes on plasmids indicates that the benefits outweigh the costs. The selective pressures imposed by diverse environments likely drive the retention of these genes on mobile genetic elements.

In summary, the abundance of the gene cluster {1898, 1088, 1209, 1091} in plasmids compared to chromosomes suggests that these genes provide selective advantages to bacteria when located on plasmids.
_________________________________________



- Related article: **O-Antigen-Dependent Colicin Insensitivity of Uropathogenic Escherichia coli**

**Citation**: Sharp C, Boinett C, Cain AHousden NG, Kumar S, Turner K, Parkhill J, Kleanthous C.2019.O-Antigen-Dependent Colicin Insensitivity of Uropathogenic Escherichia coli. J Bacteriol201:10.1128/jb.00545-18.https://doi.org/10.1128/jb.00545-18

**Summary**:
The article discusses the phenomenon of colicin insensitivity in Escherichia coli (E. coli) strains, particularly focusing on uropathogenic E. coli (UPEC) sequence type 131 (ST131). Colicins are antimicrobial proteins secreted by E. coli, and they have the potential to be used as narrow-spectrum antimicrobials.

The researchers demonstrate that this insensitivity can be overcome by making minor changes to the environmental conditions.

They found that the production of O-antigen is a key factor in colicin insensitivity but that slight modifications to growth conditions can render the organism sensitive to colicins. The reintroduction of O-antigen into E. coli K-12 demonstrated that the density of O-antigen is a dominant factor governing colicin insensitivity.

One of the measures that was used in the research: A transposon-directed insertion sequencing (TraDIS) approach was employed to identify genes associated with colicin insensitivity in UPEC ST131. Thirty-three genes (four of them are the genes in the cluster we found) were mapped to operons responsible for lipopolysaccharide (LPS) biosynthesis, O-antigen biosynthesis, and other outer membrane-related processes.

Fluorescence microscopy revealed that the presence of O-antigen diminished colicin binding to the outer membrane receptor (BtuB). Changes in O-antigen density influenced colicin binding and sensitivity.

In summary, the study provides insights into the mechanisms and environmental factors influencing colicin insensitivity in UPEC ST131, shedding light on the complex interplay between bacterial genetics, outer membrane composition, and environmental conditions.